# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
F. Walter  ->  F. Walter  |  ['F. Walter']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
Arxiv has 44 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2408.07745


extracting tarball to tmp_2408.07745...

 done.


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
Retrieving document from  https://arxiv.org/e-print/2408.08026


...

 done.
Retrieving document from  https://arxiv.org/e-print/2408.08299


extracting tarball to tmp_2408.08299...

 done.


H. Beuther  ->  H. Beuther  |  ['H. Beuther']


Found 94 bibliographic references in tmp_2408.08299/wells_2024.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.08026-b31b1b.svg)](https://arxiv.org/abs/2408.08026) | **JWST MIRI and NIRCam observations of NGC 891 and its circumgalactic medium**  |
|| J. Chastenet, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-08-16*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics; 16 pages, 8 figures*|
|**Abstract**|            We present new JWST observations of the nearby, prototypical edge-on, spiral galaxy NGC 891. The northern half of the disk was observed with NIRCam in its F150W and F277W filters. Absorption is clearly visible in the mid-plane of the F150W image, along with vertical dusty plumes that closely resemble the ones seen in the optical. A $\sim 10 \times 3~{\rm kpc}^2$ area of the lower circumgalactic medium (CGM) was mapped with MIRI F770W at 12 pc scales. Thanks to the sensitivity and resolution of JWST, we detect dust emission out to $\sim 4$ kpc from the disk, in the form of filaments, arcs, and super-bubbles. Some of these filaments can be traced back to regions with recent star formation activity, suggesting that feedback-driven galactic winds play an important role in regulating baryonic cycling. The presence of dust at these altitudes raises questions about the transport mechanisms at play and suggests that small dust grains are able to survive for several tens of million years after having been ejected by galactic winds in the disk-halo interface. We lay out several scenarios that could explain this emission: dust grains may be shielded in the outer layers of cool dense clouds expelled from the galaxy disk, and/or the emission comes from the mixing layers around these cool clumps where material from the hot gas is able to cool down and mix with these cool cloudlets. This first set of data and upcoming spectroscopy will be very helpful to understand the survival of dust grains in energetic environments, and their contribution to recycling baryonic material in the mid-plane of galaxies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.08299-b31b1b.svg)](https://arxiv.org/abs/2408.08299) | **Dynamical Accretion Flows -- ALMAGAL: Flows along filamentary structures in high-mass star-forming clusters**  |
|| M. R. A. Wells, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2024-08-16*|
|*Comments*| *11 pages, 11 figures, accepted for publication in A&A*|
|**Abstract**|            We use data from the ALMA Evolutionary Study of High Mass Protocluster Formation in the Galaxy (ALMAGAL) survey to study 100 ALMAGAL regions at $\sim$ 1\arcsec~ resolution located between $\sim$ 2 and 6~kpc distance. Using ALMAGAL $\sim$ 1.3mm line and continuum data we estimate flow rates onto individual cores. We focus specifically on flow rates along filamentary structures associated with these cores. Our primary analysis is centered around position velocity cuts in H$_2$CO (3$_{0,3}$ - 2$_{0,2}$) which allow us to measure the velocity fields, surrounding these cores. Combining this work with column density estimates we derive the flow rates along the extended filamentary structures associated with cores in these regions. We select a sample of 100 ALMAGAL regions covering four evolutionary stages from quiescent to protostellar, Young Stellar Objects (YSOs), and \HII~regions (25 each). Using dendrogram and line analysis, we identify a final sample of 182 cores in 87 regions. In this paper, we present 728 flow rates for our sample (4 per core), analysed in the context of evolutionary stage, distance from the core, and core mass. On average, for the whole sample, we derive flow rates on the order of $\sim$10$^{-4}$ M$_{sun}$yr$^{-1}$ with estimated uncertainties of $\pm$50\%. We see increasing differences in the values among evolutionary stages, most notably between the less evolved (quiescent/protostellar) and more evolved (YSO/\HII~region) sources. We also see an increasing trend as we move further away from the centre of these cores. We also find a clear relationship between the flow rates and core masses $\sim$M$^{2/3}$ which is in line with the result expected from the tidal-lobe accretion mechanism. Overall, we see increasing trends in the relationships between the flow rate and the three investigated parameters; evolutionary stage, distance from the core, and core mass.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.08114-b31b1b.svg)](https://arxiv.org/abs/2408.08114) | **Quantifying the informativity of emission lines to infer physical conditions in giant molecular clouds. I. Application to model predictions**  |
|| L. Einig, et al. -- incl., <mark>A. Hughes</mark> |
|*Appeared on*| *2024-08-16*|
|*Comments*| **|
|**Abstract**|            Observations of ionic, atomic, or molecular lines are performed to improve our understanding of the interstellar medium (ISM). However, the potential of a line to constrain the physical conditions of the ISM is difficult to assess quantitatively, because of the complexity of the ISM physics. The situation is even more complex when trying to assess which combinations of lines are the most useful. Therefore, observation campaigns usually try to observe as many lines as possible for as much time as possible. We search for a quantitative statistical criterion to evaluate the constraining power of a (or combination of) tracer(s) with respect to physical conditions in order to improve our understanding of the statistical relationships between ISM tracers and physical conditions and helps observers to motivate their observation proposals. The best tracers are obtained by comparing the mutual information between a physical parameter and different sets of lines. We apply this method to simulations of radio molecular lines emitted by a photodissociation region similar to the Horsehead Nebula that would be observed at the IRAM 30m telescope. We search for the best lines to constrain the visual extinction $A_v^{tot}$ or the far UV illumination $G_0$. The most informative lines change with the physical regime (e.g., cloud extinction). Short integration time of the CO isotopologue $J=1-0$ lines already yields much information on the total column density most regimes. The best set of lines to constrain the visual extinction does not necessarily combine the most informative individual lines. Precise constraints on $G_0$ are more difficult to achieve with molecular lines. They require spectral lines emitted at the cloud surface (e.g., [CII] and [CI] lines). This approach allows one to better explore the knowledge provided by ISM codes, and to guide future observation campaigns.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.07745-b31b1b.svg)](https://arxiv.org/abs/2408.07745) | **The Small Sizes and High Implied Densities of `Little Red Dots' with Balmer Breaks Could Explain Their Broad Emission Lines Without an AGN**  |
|| J. F. Baggen, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-08-16*|
|*Comments*| **|
|**Abstract**|            Early JWST studies found an apparent population of massive, compact galaxies at redshifts $z\gtrsim7$. Recently three of these galaxies were shown to have prominent Balmer breaks, demonstrating that their light at $\lambda_{\rm rest} \sim 3500$ $Å$ is dominated by a stellar population that is relatively old ($\sim$200 Myr). All three also have broad H$\beta$ emission with $\sigma > 1000 \,\rm km s^{-1}$, a common feature of such `little red dots'. From Sérsic profile fits to the NIRCam images in F200W we find that the stellar light of galaxies is extremely compact: the galaxies have half-light radii of $r_{\rm e}\sim$ 100 pc, in the regime of ultra compact dwarfs in the nearby Universe. Their masses are uncertain, as they depend on the contribution of possible light from an AGN to the flux at $\lambda_{\rm rest}>5000$ $Å$. If the AGN contribution is low beyond the Balmer break region, the masses are $M_* \sim 10^{10}-10^{11}\mathrm{M}_{\odot}$, and the central densities are higher than those of any other known galaxy population by an order of magnitude. Interestingly, the implied velocity dispersions of $\sim$1500 kms$^{-1}$ are in very good agreement with the measured H$\beta$ line widths. We suggest that some of the broad lines in `little red dots' are not due to AGNs but simply reflect the kinematics of the galaxies, and speculate that the galaxies are observed in a short-lived phase where the central densities are much higher than at later times. We stress, however, that the canonical interpretation of AGNs causing the broad H$\beta$ lines also remains viable.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 36</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2408.08026/./figures/paper1_nircam_tw.png', 'tmp_2408.08026/./figures/paper1_miri_tw.png', 'tmp_2408.08026/./figures/rgb_annot_multipanel.png', 'tmp_2408.08026/./figures/vertprof_nircam_vB_2.png', 'tmp_2408.08026/./figures/vertprof_nircam_optical_vB.png']
copying  tmp_2408.08026/./figures/paper1_nircam_tw.png to _build/html/
copying  tmp_2408.08026/./figures/paper1_miri_tw.png to _build/html/
copying  tmp_2408.08026/./figures/rgb_annot_multipanel.png to _build/html/
copying  tmp_2408.08026/./figures/vertprof_nircam_vB_2.png to _build/html/
copying  tmp_2408.08026/./figures/vertprof_nircam_optical_vB.png to _build/html/
exported in  _build/html/2408.08026.md
    + _build/html/tmp_2408.08026/./figures/paper1_nircam_tw.png
    + _build/html/tmp_2408.08026/./figures/paper1_miri_tw.png
    + _build/html/tmp_2408.08026/./figures/rgb_annot_multipanel.png
    + _build/html/tmp_2408.08026/./figures/vertprof_nircam_vB_2.png
    + _build/html/tmp_2408.08026/./figures/vertprof_

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\tfill}{\textbf{!fill!}\xspace}$
$\newcommand{\ssm}{7.7~\mum\xspace}$
$\newcommand{\astrodendro}{{\sc astrodendro}\xspace}$
$\newcommand{\mjysr}{MJy~sr^{-1}\xspace}$
$\newcommand{\orcid}[1]{\href{https://orcid.org/#1}{\includegraphics[width=10pt]{figures/orcid-ID.png}}}$</div>



<div id="title">

# JWST MIRI and NIRCam observations of NGC 891 and its circumgalactic medium

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.08026-b31b1b.svg)](https://arxiv.org/abs/2408.08026)<mark>Appeared on: 2024-08-16</mark> -  _Accepted for publication in Astronomy & Astrophysics; 16 pages, 8 figures_

</div>
<div id="authors">

J. Chastenet, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present new JWST observations of the nearby, prototypical edge-on, spiral galaxy NGC 891.The northern half of the disk was observed with NIRCam in its F150W and F277W filters. Absorption is clearly visible in the mid-plane of the F150W image, along with vertical dusty plumes that closely resemble the ones seen in the optical.A ${\sim }10 \times 3 {\rm kpc}^2$ area of the lower circumgalactic medium (CGM) was mapped with MIRI F770W at 12 pc scales. Thanks to the sensitivity and resolution of JWST, we detect dust emission out to $\sim 4$ kpc from the disk, in the form of filaments, arcs, and super-bubbles. Some of these filaments can be traced back to regions with recent star formation activity, suggesting that feedback-driven galactic winds play an important role in regulating baryonic cycling.The presence of dust at these altitudes raises questions about the transport mechanisms at play and suggests that small dust grains are able to survive for several tens of million years after having been ejected by galactic winds in the disk-halo interface. We lay out several scenarios that could explain this emission: dust grains may be shielded in the outer layers of cool dense clouds expelled from the galaxy disk, and/or the emission comes from the mixing layers around these cool clumps where material from the hot gas is able to cool down and mix with these cool cloudlets.This first set of data and upcoming spectroscopy will be very helpful to understand the survival of dust grains in energetic environments, and their contribution to recycling baryonic material in the mid-plane of galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2408.08026/./figures/paper1_nircam_tw.png" alt="Fig5.1" width="50%"/><img src="tmp_2408.08026/./figures/paper1_miri_tw.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** _Top left:_ NIRCam F150W data, at native $\sim 0.05"$ resolution. The mid-plane shows conspicuous absorption. The background in that band shows large discrepancies, due to the difficulty to correct for both large structure stripes and offsets between the individual dithers. The small squares are gaps in the dither-pattern coverage.
    _Top right:_ NIRCam F277W data, at native $\sim 0.092"$ resolution. Some absorption is visible in the mid-plane, and the image shows a much smoother background than for the F150W mosaic.
    _Bottom:_ MIRI F770W data, at native $\sim 0.25"$ resolution, with radial distance indicators at 1, 2, 5, and 10 kpc from the mid-plane. The mid-IR image shows many filamentary structures as far out as $\sim 4$ kpc. (*fig:finalimages*)

</div>
<div id="div_fig2">

<img src="tmp_2408.08026/./figures/rgb_annot_multipanel.png" alt="Fig7" width="100%"/>

**Figure 7. -** Multi-colour image with red: F277W, green: F150W, and blue: V-band, showing the coverage where all data overlap (the V-band and F277W images were adjusted to show the same spatial coverage, including dither gaps), and individual images shown in insets on the right. The NIRCam data was convolved to a $0.2"$ Gaussian PSF.
    We indicate galactocentric radii $r$ at 3 and 7 kpc.
    Conspicuous dark features show dusty filaments seen in extinction perpendicular to the disk. We can also notice the presence of star clusters, some obscured by dust, and some are not. (*fig:RGB*)

</div>
<div id="div_fig3">

<img src="tmp_2408.08026/./figures/vertprof_nircam_vB_2.png" alt="Fig1.1" width="50%"/><img src="tmp_2408.08026/./figures/vertprof_nircam_optical_vB.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** _Top:_ NIRCam vertical profiles shown with symbols, in light blue for F150W and dark blue for F277W. We also show the fit from [Fraternali, Sancisi and Kamphuis (2011)]() using IRAC data (black line) and two fits from the radiative transfer models of \citet[][]{Xilouris1998}. In all cases, we average the vertical profiles over galactocentric radii $2.5 {\rm kpc}\leq r \leq 9 {\rm kpc}$, so that the radial coverage is the same.
    _Bottom:_ Emission profiles across the disk at $r \sim 8 $kpc showing the NIRCam data, and three optical bands. The images have all been normalised to the average values at $z = 1 $kpc, due to different units, for an easier comparison.
    The upturn at 1.2 kpc in F150W is an artefact due to one of the dither offsets mentioned in the text. (*fig:NIRCamVP*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.08026"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\HII}{H{\sc ii}}$
$\newcommand{\flowrate}{10^{-4}~M_{\sun}\mathrm{yr}^{-1}}$</div>



<div id="title">

# Dynamical Accretion Flows

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.08299-b31b1b.svg)](https://arxiv.org/abs/2408.08299)<mark>Appeared on: 2024-08-16</mark> -  _11 pages, 11 figures, accepted for publication in A&A_

</div>
<div id="authors">

M. R. A. Wells, et al. -- incl., <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** Investigation of the flow of material along filamentary structures towards the central core can help provide insight into high-mass star formation and evolution. Our main motivation is to answer the question: what are the properties of accretion flows in star-forming clusters? We use data from the ALMA Evolutionary Study of High Mass Protocluster Formation in the Galaxy (ALMAGAL) survey to study 100 ALMAGAL regions at $\sim$ 1 $\arcsec$ resolution located between $\sim$ 2 and 6 kpc distance. Making use of the ALMAGAL $\sim$ 1.3mm line and continuum data we estimate flow rates onto individual cores. We focus specifically on flow rates along filamentary structures associated with these cores. Our primary analysis is centered around position velocity cuts in $H_2$ CO (3 $_{0,3}$ - 2 $_{0,2}$ ) which allow us to measure the velocity fields, surrounding these cores. Combining this work with column density estimates we derive the flow rates along the extended filamentary structures associated with cores in these regions. We select a sample of 100 ALMAGAL regions covering four evolutionary stages from quiescent to protostellar, Young Stellar Objects (YSOs), and $\HII$ regions (25 each). Using dendrogram and line analysis, we identify a final sample of 182 cores in 87 regions. In this paper, we present 728 flow rates for our sample (4 per core), analysed in the context of evolutionary stage, distance from the core, and core mass. On average, for the whole sample, we derive flow rates on the order of $\sim$ $\flowrate$ with estimated uncertainties of $\pm$ 50 \% . We see increasing differences in the values among evolutionary stages, most notably between the less evolved (quiescent/protostellar) and more evolved (YSO/ $\HII$ region) sources and we also see an increasing trend as we move further away from the centre of these cores. We also find a clear relationship between the calculated flow rates and core masses $\sim$ M $^{2/3}$ which is in line with the result expected from the tidal-lobe accretion mechanism. The significance of these relationships is tested with Kolmogorov-Smirnov and Mann-Whitney U tests. Overall, we see increasing trends in the relationships between the flow rate and the three investigated parameters; evolutionary stage, distance from the core, and core mass.

</div>

<div id="div_fig1">

<img src="tmp_2408.08299/./images/core_mass.png" alt="Fig8" width="100%"/>

**Figure 8. -** Scatter plot of the results of the whole sample showing flow rate vs. core mass in grey. The purple points are the average flow rate/mass values per bin, with the associated errors. Here, each bin contains the same number of cores. A line of best fit is shown in black. (*fig:coremass*)

</div>
<div id="div_fig2">

<img src="tmp_2408.08299/./images/880433_SO.png" alt="Fig3" width="100%"/>

**Figure 3. -** 0th moment map of SO ($6_5$ - $5_4$) in grey-scale for source AG348.5792-0.9197 overlaid with continuum contours in black (levels 3,6,9 $\sigma_{cont}$). A green star to show the peak intensity position of the core. Red and blue contours show the "wings" of the spectral line emission, from 3 to 20 kms$^{-1}$ either side, with respect to the region velocity of rest. (*fig:outflow*)

</div>
<div id="div_fig3">

<img src="tmp_2408.08299/./images/molly_xy_pdf.png" alt="Fig1" width="100%"/>

**Figure 1. -** Source distribution for the regions in our ALMAGAL sub-sample is shown as black dots. The size of the markers scales with the masses of the ALMAGAL clumps. Grey dots are the rest of the ALMAGAL survey and the dashed line is a heliocentric distance circle at 5 kpc. (*fig:survey*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.08299"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

163  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
